In [0]:
# LIBRARIES
import gensim 
import logging
import re

!pip install -q unidecode
import unidecode

!pip install -q nltk
import nltk
#nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

!pip install -q num2words
import num2words as n2w


# Config logs
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
# Know the current directory
!pwd

/content


In [3]:
# Access to Google Drive
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 20.6MB/s 


In [4]:
# Mount Google Drive Unit in /content
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
#Upload Data from Google Drive
filename = "eswiki-latest-pages-articles.xml.bz2"
link = 'https://drive.google.com/open?id=1DPZZQ43w8brRhbEMolgLqOWKbZbE-IQu' # The shareable link
fluff, id = link.split('=')
print ("File ID", id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile(filename)

File ID 1DPZZQ43w8brRhbEMolgLqOWKbZbE-IQu


AttributeError: ignored

In [0]:
# Upload Dataset from Local
from google.colab import files
uploaded = files.upload()

In [15]:
# Create a corpus from Wikipedia dump file
"""

Inspired by:
https://github.com/panyang/Wikipedia_Word2vec/blob/master/v1/process_wiki.py
"""

import sys
from gensim.corpora import WikiCorpus

def make_corpus(in_f, out_f):

	"""Convert Wikipedia xml dump file to text corpus"""

	output = open(out_f, 'w')
	wiki = WikiCorpus(in_f)

	i = 0
	for text in wiki.get_texts():
		output.write(bytes(' '.join(text), 'utf-8').decode('utf-8') + '\n')
		i = i + 1
		if (i % 10000 == 0):
			print('Processed ' + str(i) + ' articles')
	output.close()
	print('Processing complete!')

# CREATE CORPUS .txt FROM WIKIPEDIA ES

path = '/content/gdrive/My Drive'
in_f = "eswiki-latest-pages-articles-multistream.xml.bz2"
out_f = "es_wiki.txt"
make_corpus(path+'/'+in_f, path+'/'+out_f)



2019-04-10 10:56:18,385 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-04-10 10:59:36,713 : INFO : adding document #10000 to Dictionary(399581 unique tokens: ['abedules', 'abierto', 'abril', 'abrir', 'absoluta']...)
2019-04-10 11:02:20,145 : INFO : adding document #20000 to Dictionary(562023 unique tokens: ['abedules', 'abierto', 'abril', 'abrir', 'absoluta']...)
2019-04-10 11:04:49,315 : INFO : adding document #30000 to Dictionary(694992 unique tokens: ['abedules', 'abierto', 'abril', 'abrir', 'absoluta']...)
2019-04-10 11:07:14,070 : INFO : adding document #40000 to Dictionary(805674 unique tokens: ['abedules', 'abierto', 'abril', 'abrir', 'absoluta']...)
2019-04-10 11:09:23,023 : INFO : adding document #50000 to Dictionary(896502 unique tokens: ['abedules', 'abierto', 'abril', 'abrir', 'absoluta']...)
2019-04-10 11:11:14,828 : INFO : adding document #60000 to Dictionary(969674 unique tokens: ['abedules', 'abierto', 'abril', 'abrir', 'absoluta']...)
2019-04-10 11

KeyboardInterrupt: ignored

In [0]:
# READ CORPUS
file = open("es_wiki.txt", 'r', encoding="utf-8")
txt_raw = file.read()
file.close()
#print(txt_raw)

# Save corpus in Google Drive Unit
#path_gd = '/content/gdrive'
#folder_fg = ''
#filename = 'es_wiki.txt'
#file = open(filename, 'w') 
#file.write(txt_raw)
#file.close()


In [0]:
# PREPROCESSING - PART I

# Replace "\n" and "-" with space
txt_aux2 = txt_aux.replace("-", " ")
txt_aux = txt_aux2
#print(txt_aux)

#Remove accents and punctuation
txt_aux = unidecode.unidecode(txt_aux)
#txt_aux = re.sub(r'[^a-zA-Z0-9]', r' ', txt_aux) 
#####Hay que dejarle la puntuación para que divida en frases!!

#Tokenizing, numbers to words and lowercasing
data_aux = []
for i in sent_tokenize(txt_aux):
  temp = []
  # tokenize the sentence into words
  for j in word_tokenize(i):
    if j.isdigit():
      j = n2w.num2words(int(j), lang="es")

    temp.append(j.lower())

  data_aux.append(temp)
  
print("Len(data_aux): ", len(data_aux))
#print(data_aux[:3])


In [0]:
# PREPROCESSING - PART II
def remove_blanks(tokens):
  return [token.strip() for token in tokens]
  
def remove_list_nulls(tokens):
  return [token for token in tokens if token != '']
    
  
def remove_punctuation(tokens):
  return [re.sub(r'[^a-zA-Z0-9]', r' ', token) for token in tokens]

data = []
for i in data_aux:
  tokens = remove_punctuation(i)
  tokens = remove_blanks(tokens)
  tokens = remove_list_nulls(tokens)
  data.append(tokens)

print(len(data))
print(*data[:10], sep="\n")


In [0]:
# DEFINE THE WORD2VEC MODEL
documents = data
model = gensim.models.Word2Vec(documents, 
                               size=150, 
                               window=10, 
                               min_count=2, 
                               workers=10,
                               sg=0)

In [0]:
# TRAIN THE MODEL
model.train(documents,total_examples=len(documents),epochs=10)

In [0]:
# TRYINGS
w1 = "madrid"
model.wv.most_similar(positive=w1)

In [0]:
# SAVE THE MODEL IN GOOGLE DRIVE
 model.save("word2vec_wiki_es.model")
model_file = drive.CreateFile({'title' : 'word2vec_wiki_es.model'})                       
model_file.SetContentFile('model.h5')                       
model_file.Upload()